In [1]:
%%capture
#!conda install -c conda-forge gdal pyarrow

# Data Processing and Feature Engineering Pipeline

**Objective:** This notebook details the steps to preprocess raw data, handle missing values, perform feature engineering, and prepare a cleaned dataset for further analysis or modeling.

**Sections:**
1.  Initial Data Loading and Type Inspection
2.  Date Column Processing and Feature Creation
3.  Handling Missing Values in Date Columns
4.  Processing Boolean Flag Columns (FL_)
5.  Processing Categorical Code Columns (CD_)
6.  Processing General Text Columns (GN_)
7.  Processing Numerical Columns (NM_)
8.  Feature Engineering from Existing Data
9.  Final Data Cleaning and Validation
10. Saving Processed Data (Optional)

## 1. Initial Data Loading and Type Inspection
This section loads the raw dataset and performs an initial inspection of data types. The original data types are also saved to a CSV for reference.

In [2]:
import pandas as pd
import numpy as np

# Load the raw dataset
# low_memory=False is used to ensure correct type inference for mixed-type columns
df = pd.read_csv("data/raw_data.csv", low_memory=False)

# Display initial data type distribution
print("Initial data types summary:")
print(df.dtypes.value_counts())  #

# Save the original data types to a CSV file for documentation purposes
df.dtypes.reset_index().rename(columns={"index": "Column Name", 0: "Data Type"}).to_csv(
    "./data/original_data_types.csv", index=False
)  #
print("\nOriginal data types saved to './data/original_data_types.csv'")

Initial data types summary:
object     133
int64       31
float64     14
Name: count, dtype: int64

Original data types saved to './data/original_data_types.csv'


## 2. Date Column Processing and Feature Creation
This section focuses on columns representing dates. It involves:
* Converting relevant columns to datetime objects.
* Filtering the DataFrame based on non-missing key date fields.
* Calculating delivery delay as a preliminary feature.

In [3]:
# Identify columns that represent dates (conventionally prefixed with "DT_")
date_cols = df.columns[df.columns.str.startswith("DT_")]  #

# Convert these columns to datetime objects
# 'errors="coerce"' will turn unparseable dates into NaT (Not a Time)
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors="coerce")  #
print(f"Converted {len(date_cols)} columns to datetime objects.")

Converted 61 columns to datetime objects.


In [4]:
# Filter the DataFrame to include only rows where key delivery dates are present
# This is crucial for calculating delays and other time-based features accurately.
# Using .copy() to avoid SettingWithCopyWarning and ensure 'filtered_df' is a new DataFrame.
filtered_df = df[
    df["DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE"].notna()
    & df["DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE"].notna()
].copy()  #

print(f"Shape of DataFrame after filtering for essential dates: {filtered_df.shape}")

Shape of DataFrame after filtering for essential dates: (10027, 178)


In [5]:
# Calculate the delivery delay in days
# This is the difference between actual arrival and expected delivery.
filtered_df["Delay_days"] = (
    filtered_df["DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE"]
    - filtered_df["DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE"]
)  #
print("Calculated 'Delay_days'.")

Calculated 'Delay_days'.


In [6]:
# Verify the shape of the filtered DataFrame
print(f"Current shape of filtered_df: {filtered_df.shape}")  #

Current shape of filtered_df: (10027, 179)


In [7]:
# Analyze the calculated delays
# A positive delay means the order arrived after the expected date.
order_delayed_boolean_series = filtered_df["Delay_days"].dt.days > 0  #

# Count the number of orders that were delayed
num_order_delayed = order_delayed_boolean_series.sum()  #

# Calculate the percentage of delayed orders
percentage_delayed = (
    (num_order_delayed / len(filtered_df)) * 100 if len(filtered_df) > 0 else 0
)  #

print(f"Number of orders delayed: {num_order_delayed}")
print(f"Percentage of orders delayed: {percentage_delayed:.2f}%")

Number of orders delayed: 4819
Percentage of orders delayed: 48.06%


### 2.1 Selecting and Dropping Date Columns
Keep only essential date columns for further processing and feature engineering. Other date columns are dropped.

In [8]:
# Define the list of date columns to retain for analysis
columns_to_keep = [
    "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE",
    "DT_VEHICLE_FACTORY_PRODUCTION_DATE",
    "DT_VEHICLE_PASSED_TO_SALES_DATE",
    "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE",
    "DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE",
    "DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE",
    "DT_CUSTOMS_OFFICE_INBOUND_DATE",
    "DT_CUSTOMS_OFFICE_OUTBOUND_DATE",
    "DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE",
]  #

In [9]:
# Re-identify all date columns currently in filtered_df (prefixed with "DT_")
current_date_cols = filtered_df.columns[filtered_df.columns.str.startswith("DT_")]

# Determine which date columns to drop
date_cols_to_drop = [col for col in current_date_cols if col not in columns_to_keep]

if date_cols_to_drop:
    filtered_df.drop(columns=date_cols_to_drop, inplace=True)  #
    print(f"Dropped {len(date_cols_to_drop)} non-essential date columns.")
else:
    print("No date columns to drop based on the keep list.")

# Update the list of active date columns after dropping
date_cols = filtered_df.columns[filtered_df.columns.str.startswith("DT_")]  #
print(f"Remaining date columns: {date_cols.tolist()}")

Dropped 52 non-essential date columns.
Remaining date columns: ['DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE', 'DT_VEHICLE_FACTORY_PRODUCTION_DATE', 'DT_VEHICLE_PASSED_TO_SALES_DATE', 'DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE', 'DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE', 'DT_CUSTOMS_OFFICE_INBOUND_DATE', 'DT_CUSTOMS_OFFICE_OUTBOUND_DATE', 'DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE', 'DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE']


In [10]:
# Check for missing values in the remaining date columns
print(
    "\nMissing values in remaining date columns after initial filtering and selection:"
)
print(filtered_df[date_cols].isna().sum())  #


Missing values in remaining date columns after initial filtering and selection:
DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE                    52
DT_VEHICLE_FACTORY_PRODUCTION_DATE                    7877
DT_VEHICLE_PASSED_TO_SALES_DATE                         52
DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE     307
DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE       5257
DT_CUSTOMS_OFFICE_INBOUND_DATE                        7593
DT_CUSTOMS_OFFICE_OUTBOUND_DATE                       9190
DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE            0
DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE              0
dtype: int64


## 3. Handling Missing Values in Date Columns
This section addresses missing (NaT) values in critical date columns. Strategies include:
* Dropping rows where key chronological dates are missing.
* Imputing other missing dates based on median time differences from related, non-missing dates.
* Creating flag columns to indicate where imputation occurred.

In [11]:
# Drop rows if 'DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE' or 'DT_VEHICLE_PASSED_TO_SALES_DATE' are missing,
# as these are fundamental for many downstream calculations and imputations.
initial_rows = len(filtered_df)
filtered_df.dropna(
    subset=["DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE", "DT_VEHICLE_PASSED_TO_SALES_DATE"],
    inplace=True,
)  #
rows_dropped = initial_rows - len(filtered_df)
print(f"Dropped {rows_dropped} rows due to missing key order/sales dates.")
print(
    f"Shape of DataFrame after dropping rows with missing essential dates: {filtered_df.shape}"
)

Dropped 52 rows due to missing key order/sales dates.
Shape of DataFrame after dropping rows with missing essential dates: (9975, 127)


### 3.1 Imputing `DT_VEHICLE_FACTORY_PRODUCTION_DATE`

In [12]:
# Calculate the median production time (days from order to production) for non-missing rows
production_time_series = (
    filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"].dropna()
    - filtered_df.loc[
        filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"].notna(),
        "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE",
    ]
).dt.days  #

median_production_time_days = production_time_series.median()  #
print(f"Median production time: {median_production_time_days} days.")

Median production time: 81.0 days.


In [13]:
# Create a flag column to indicate if the production date was originally missing (and thus imputed)
filtered_df["MISSING_PRODUCTION_DATE_FLAG"] = (
    filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"].isna().astype(int)
)  #
print(
    f"{filtered_df['MISSING_PRODUCTION_DATE_FLAG'].sum()} production dates will be imputed."
)

7825 production dates will be imputed.


In [14]:
# Impute missing production dates by adding the median production time to the order entry date
if pd.notna(median_production_time_days):
    imputation_values = filtered_df[
        "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"
    ] + pd.Timedelta(days=median_production_time_days)
    filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"] = filtered_df[
        "DT_VEHICLE_FACTORY_PRODUCTION_DATE"
    ].fillna(imputation_values)  #
    print("Imputed missing 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' values.")
else:
    print(
        "Median production time is NaN, skipping imputation for 'DT_VEHICLE_FACTORY_PRODUCTION_DATE'."
    )

Imputed missing 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' values.


In [15]:
# Validate imputation: Check if any imputed production dates are earlier than their order dates
invalid_imputed_rows = filtered_df[
    (filtered_df["MISSING_PRODUCTION_DATE_FLAG"] == 1)
    & (
        filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"]
        < filtered_df["DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"]
    )
]  #
print(
    f"Number of rows where imputed production date is before order date: {len(invalid_imputed_rows)}"
)
# Further investigation or adjustment might be needed if this count is high.

Number of rows where imputed production date is before order date: 0


In [16]:
# Verify that all NaNs in 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' have been handled
print(
    f"Missing values in 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' after imputation: {filtered_df['DT_VEHICLE_FACTORY_PRODUCTION_DATE'].isna().sum()}"
)  #

Missing values in 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' after imputation: 0


### 3.2 Imputing `DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE`

In [17]:
# Calculate the median lag time from production to 'ready to ship'
ready_to_ship_lag_series = (
    filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"].dropna()
    - filtered_df.loc[
        filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"].notna(),
        "DT_VEHICLE_FACTORY_PRODUCTION_DATE",
    ]
).dt.days  #

median_ready_to_ship_lag_days = ready_to_ship_lag_series.median()  #
print(
    f"Median lag from production to ready-to-ship: {median_ready_to_ship_lag_days} days."
)

Median lag from production to ready-to-ship: -12.0 days.


In [18]:
# Create a flag for missing 'ready to ship' dates
filtered_df["MISSING_READY_TO_SHIP_FLAG"] = (
    filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"].isna().astype(int)
)  #
print(
    f"{filtered_df['MISSING_READY_TO_SHIP_FLAG'].sum()} 'ready to ship' dates will be imputed."
)

255 'ready to ship' dates will be imputed.


In [19]:
# Impute missing 'ready to ship' dates using the median lag from the (now imputed) production date
if pd.notna(median_ready_to_ship_lag_days):
    imputation_values_rts = filtered_df[
        "DT_VEHICLE_FACTORY_PRODUCTION_DATE"
    ] + pd.Timedelta(days=median_ready_to_ship_lag_days)
    filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"] = filtered_df[
        "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"
    ].fillna(imputation_values_rts)  #
    print(
        "Imputed missing 'DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE' values."
    )
else:
    print("Median ready-to-ship lag is NaN, skipping imputation.")

Imputed missing 'DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE' values.


In [20]:
# Verify imputation
print(
    f"Missing values in 'DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE' after imputation: {filtered_df['DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE'].isna().sum()}"
)  #

Missing values in 'DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE' after imputation: 0


### 3.3 Imputing `DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE`

In [21]:
# Calculate the 90th percentile lag from 'ready to ship' to 'shipping order creation'
# Using a higher percentile (e.g., 90th) can be a strategy for more conservative estimation if quick turnarounds are outliers.
shipping_order_lag_series = (
    filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"].dropna()
    - filtered_df.loc[
        filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"].notna(),
        "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE",
    ]
).dt.days  #

# Calculate the 90th percentile of this lag
percentile_90_shipping_order_lag_days = shipping_order_lag_series.quantile(0.9)  #
print(
    f"90th percentile lag from ready-to-ship to shipping order creation: {percentile_90_shipping_order_lag_days} days."
)

90th percentile lag from ready-to-ship to shipping order creation: 7.0 days.


In [22]:
# Flag missing shipping order dates
filtered_df["MISSING_SHIPPING_ORDER_FLAG"] = (
    filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"].isna().astype(int)
)  #
print(
    f"{filtered_df['MISSING_SHIPPING_ORDER_FLAG'].sum()} 'shipping order creation' dates will be imputed."
)

5205 'shipping order creation' dates will be imputed.


In [23]:
# Impute missing shipping order dates using the 90th percentile lag
if pd.notna(percentile_90_shipping_order_lag_days):
    imputation_values_so = filtered_df[
        "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"
    ] + pd.Timedelta(days=percentile_90_shipping_order_lag_days)
    filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"] = filtered_df[
        "DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"
    ].fillna(imputation_values_so)  #
    print("Imputed missing 'DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE' values.")
else:
    print("90th percentile shipping order lag is NaN, skipping imputation.")

Imputed missing 'DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE' values.


In [24]:
# Validate imputation: Check for rows where shipping order date is before ready-to-ship date
# This could happen if the lag used for imputation is negative or too small, or due to original data issues.
invalid_shipping_order_rows = filtered_df[
    filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"]
    < filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"]
]  #
print(
    f"Number of rows where imputed shipping order date is before ready-to-ship date: {len(invalid_shipping_order_rows)}"
)

Number of rows where imputed shipping order date is before ready-to-ship date: 505


In [25]:
# Flag these chronologically invalid rows
filtered_df["INVALID_SHIPPING_ORDER_TIMING_FLAG"] = (
    filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"]
    < filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"]
).astype(int)  #

In [26]:
# Analyze the extent of invalid rows
invalid_timing_count = filtered_df["INVALID_SHIPPING_ORDER_TIMING_FLAG"].sum()  #
percentage_invalid_timing = (
    (invalid_timing_count / len(filtered_df)) * 100 if len(filtered_df) > 0 else 0
)
print(f"Number of rows with invalid shipping order timing: {invalid_timing_count}")
print(f"Percentage of dataset: {percentage_invalid_timing:.2f}%")

Number of rows with invalid shipping order timing: 505
Percentage of dataset: 5.06%


In [27]:
# Remove rows with invalid shipping order timing as they represent a data inconsistency
initial_rows_before_timing_drop = len(filtered_df)
filtered_df = filtered_df[
    filtered_df["INVALID_SHIPPING_ORDER_TIMING_FLAG"] == 0
].copy()  #
rows_dropped_timing = initial_rows_before_timing_drop - len(filtered_df)
print(f"Dropped {rows_dropped_timing} rows due to invalid shipping order timing.")

# Drop the flag column as it's no longer needed after filtering
filtered_df.drop(columns=["INVALID_SHIPPING_ORDER_TIMING_FLAG"], inplace=True)  #
print(
    f"Shape of DataFrame after handling invalid shipping order timing: {filtered_df.shape}"
)

Dropped 505 rows due to invalid shipping order timing.
Shape of DataFrame after handling invalid shipping order timing: (9470, 130)


In [28]:
# Display a sample of the processed date columns and their current state
print("\nSample of date columns after imputation and cleaning steps:")
if not filtered_df.empty:
    print(filtered_df[date_cols].head())  #
    print("\nMissing values check for date columns after all date imputations:")
    print(filtered_df[date_cols].isna().sum())
else:
    print("DataFrame is empty after filtering steps.")


Sample of date columns after imputation and cleaning steps:
    DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE DT_VEHICLE_FACTORY_PRODUCTION_DATE  \
210                           2024-03-29                         2024-06-18   
693                           2023-03-31                         2023-06-20   
734                           2023-06-06                         2023-08-26   
750                           2022-06-02                         2022-08-22   
801                           2024-09-10                         2024-11-30   

    DT_VEHICLE_PASSED_TO_SALES_DATE  \
210                      2024-05-21   
693                      2023-04-13   
734                      2023-10-04   
750                      2022-07-05   
801                      2024-10-10   

    DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE  \
210                                         2024-05-21   
693                                         2023-04-13   
734                                         2023-10-04 

### 3.4 Handling Customs Date Columns (`DT_CUSTOMS_OFFICE_INBOUND_DATE` & `DT_CUSTOMS_OFFICE_OUTBOUND_DATE`)
Customs dates are only relevant for international shipments. We will:
* Create a flag `IS_INTERNATIONAL` based on the presence of `DT_CUSTOMS_OFFICE_INBOUND_DATE`.
* Impute `DT_CUSTOMS_OFFICE_OUTBOUND_DATE` based on the average clearance time for international shipments where both dates are present.
* Create a status column `International_Status`.

In [29]:
# Create 'IS_INTERNATIONAL' flag: 1 if customs inbound date is present, 0 otherwise.
filtered_df["IS_INTERNATIONAL"] = (
    filtered_df["DT_CUSTOMS_OFFICE_INBOUND_DATE"].notna().astype(int)
)  #
print("'IS_INTERNATIONAL' flag created.")
print(filtered_df["IS_INTERNATIONAL"].value_counts(dropna=False))  #

'IS_INTERNATIONAL' flag created.
IS_INTERNATIONAL
0    7109
1    2361
Name: count, dtype: int64


In [30]:
# Calculate the average customs clearance time in days for shipments where both dates are available
mask_both_customs_dates_present = (
    filtered_df["DT_CUSTOMS_OFFICE_INBOUND_DATE"].notna()
    & filtered_df["DT_CUSTOMS_OFFICE_OUTBOUND_DATE"].notna()
)  #

average_customs_clearance_days = 0  # Default fallback
if mask_both_customs_dates_present.any():
    customs_clearance_times = (
        filtered_df.loc[
            mask_both_customs_dates_present, "DT_CUSTOMS_OFFICE_OUTBOUND_DATE"
        ]
        - filtered_df.loc[
            mask_both_customs_dates_present, "DT_CUSTOMS_OFFICE_INBOUND_DATE"
        ]
    ).dt.days  #
    average_customs_clearance_days = customs_clearance_times.mean()  #
    # Ensure average days is not negative, which would be illogical
    if average_customs_clearance_days < 0:
        average_customs_clearance_days = 0  # Or handle as an error/warning
        print(
            f"Warning: Calculated average customs clearance days is negative ({average_customs_clearance_days}). Setting to 0."
        )

print(
    f"Average customs clearance time: {average_customs_clearance_days:.2f} days (used for imputation if outbound date is missing but inbound exists)."
)

Average customs clearance time: 7.39 days (used for imputation if outbound date is missing but inbound exists).


In [31]:
# Impute missing 'DT_CUSTOMS_OFFICE_OUTBOUND_DATE' where 'DT_CUSTOMS_OFFICE_INBOUND_DATE' exists
mask_inbound_present_outbound_missing = (
    filtered_df["DT_CUSTOMS_OFFICE_INBOUND_DATE"].notna()
    & filtered_df["DT_CUSTOMS_OFFICE_OUTBOUND_DATE"].isna()
)  #

if mask_inbound_present_outbound_missing.any():
    imputed_outbound_dates = filtered_df.loc[
        mask_inbound_present_outbound_missing, "DT_CUSTOMS_OFFICE_INBOUND_DATE"
    ] + pd.to_timedelta(average_customs_clearance_days, unit="D")  #
    filtered_df.loc[
        mask_inbound_present_outbound_missing, "DT_CUSTOMS_OFFICE_OUTBOUND_DATE"
    ] = imputed_outbound_dates
    print(
        f"Imputed {mask_inbound_present_outbound_missing.sum()} missing 'DT_CUSTOMS_OFFICE_OUTBOUND_DATE' values."
    )
else:
    print(
        "No 'DT_CUSTOMS_OFFICE_OUTBOUND_DATE' values to impute based on existing inbound dates."
    )

Imputed 1555 missing 'DT_CUSTOMS_OFFICE_OUTBOUND_DATE' values.


In [32]:
# Create 'International_Status' column for easier interpretation
filtered_df["International_Status"] = "International (Customs Data Present)"  #

# Label rows as "Domestic / No Customs Data" if 'DT_CUSTOMS_OFFICE_INBOUND_DATE' was originally NaN
# This uses the IS_INTERNATIONAL flag created *before* imputation of outbound dates
filtered_df.loc[filtered_df["IS_INTERNATIONAL"] == 0, "International_Status"] = (
    "Domestic / No Customs Data"  #
)
print("\n'International_Status' column created:")
print(filtered_df["International_Status"].value_counts(dropna=False))


'International_Status' column created:
International_Status
Domestic / No Customs Data              7109
International (Customs Data Present)    2361
Name: count, dtype: int64


In [33]:
# Final check of NaNs in date columns after all imputation steps
print("\nMissing values in date columns after all imputation and handling:")
if not filtered_df.empty:
    print(filtered_df[date_cols].isna().sum())  #
    # Remaining NaNs in customs columns are expected for non-international shipments.
else:
    print("DataFrame is empty.")


Missing values in date columns after all imputation and handling:
DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE                     0
DT_VEHICLE_FACTORY_PRODUCTION_DATE                       0
DT_VEHICLE_PASSED_TO_SALES_DATE                          0
DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE       0
DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE          0
DT_CUSTOMS_OFFICE_INBOUND_DATE                        7109
DT_CUSTOMS_OFFICE_OUTBOUND_DATE                       7108
DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE            0
DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE              0
dtype: int64


## 4. Processing Boolean Flag Columns (FL_)
This section handles columns prefixed with `FL_`, which are typically boolean flags (0 or 1).
Steps include:
* Identifying `FL_` columns.
* Dropping specified unnecessary flag columns.
* Filling missing values (NaNs) in `FL_IS_DIRECT_SALES_FLAG` with its mode.
* Converting all remaining `FL_` columns to integer type (0 or 1).

In [34]:
# Inspect unique values of a sample FL_ column to understand its structure (e.g., NaN, 0.0, 1.0)
if "FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG" in df.columns:
    print(
        "Unique values in 'FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG' (original df):"
    )
    print(df["FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG"].unique())  #
else:
    print(
        "'FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG' not found in original df."
    )

Unique values in 'FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG' (original df):
[nan  0.  1.]


In [35]:
# Identify all columns starting with "FL_" in the filtered DataFrame
fl_cols_initial = filtered_df.columns[
    filtered_df.columns.str.startswith("FL_")
].tolist()  #
print(f"Found {len(fl_cols_initial)} columns starting with 'FL_': {fl_cols_initial}")

# Define FL_ columns to be excluded/dropped
excluded_fl_cols = [
    "FL_IS_VEHICLE_IN_MARKET_COMPOUND_INVOICEABLE_FLAG",
    # Add other FL_ columns to drop if identified as unnecessary
]  #

# Drop the excluded columns if they exist in the DataFrame
actual_fl_cols_to_drop = [col for col in excluded_fl_cols if col in filtered_df.columns]
if actual_fl_cols_to_drop:
    filtered_df.drop(columns=actual_fl_cols_to_drop, inplace=True)  #
    print(f"Dropped FL_ columns: {actual_fl_cols_to_drop}")

# Update the list of FL_ columns to process
fl_cols_to_process = [
    col for col in fl_cols_initial if col not in actual_fl_cols_to_drop
]  #

# Handle missing values specifically for 'FL_IS_DIRECT_SALES_FLAG' by filling with mode
direct_sales_col = "FL_IS_DIRECT_SALES_FLAG"
if (
    direct_sales_col in fl_cols_to_process
    and filtered_df[direct_sales_col].isna().any()
):
    mode_direct_sales = filtered_df[direct_sales_col].mode()
    if not mode_direct_sales.empty:
        filtered_df[direct_sales_col].fillna(mode_direct_sales[0], inplace=True)  #
        print(
            f"Filled NaNs in '{direct_sales_col}' with its mode ({mode_direct_sales[0]})."
        )
    else:
        # Fallback if mode cannot be determined (e.g., all NaNs), fill with 0
        filtered_df[direct_sales_col].fillna(0, inplace=True)
        print(f"Could not determine mode for '{direct_sales_col}', filled NaNs with 0.")

# For any other FL_ columns that might still have NaNs, a general fill (e.g., with 0) might be needed.
# Here, we assume they should be convertible to int, implying NaNs should be handled.
# For simplicity, let's fill remaining NaNs in FL_ columns with 0 beforeastype(int)
for col in fl_cols_to_process:
    if filtered_df[col].isna().any():
        filtered_df[col].fillna(0, inplace=True)  # General fallback for other FL_ NaNs
        print(f"Filled remaining NaNs in '{col}' with 0.")

# Convert all processed FL_ columns to integer type (0 or 1)
if fl_cols_to_process:
    filtered_df[fl_cols_to_process] = filtered_df[fl_cols_to_process].astype(int)  #
    print(f"Converted {len(fl_cols_to_process)} FL_ columns to integer type.")
    # print("Data types of FL_ columns after processing:")
    # print(filtered_df[fl_cols_to_process].dtypes)
else:
    print("No FL_ columns to process.")

Found 32 columns starting with 'FL_': ['FL_IS_FIRST_ORDER_INGESTION_FLAG', 'FL_IS_ORDER_ACTIVE_FLAG', 'FL_IS_NEW_VEHICLE_ORDER_DAMAGED_FLAG', 'FL_IS_NEW_VEHICLE_ORDER_IN_NETWORK_FLAG', 'FL_IS_NEW_VEHICLE_ORDER_IN_TRANSPORT_TO_DEALER_FLAG', 'FL_IS_NEW_VEHICLE_ORDER_PRODUCED_FLAG', 'FL_IS_NEW_VEHICLE_ORDER_STORED_FLAG', 'FL_IS_OWNED_NETWORK_SUBJECT_FLAG', 'FL_IS_FACTORY_ORDER_CODE_AVAILABLE_FLAG', 'FL_IS_FINAL_CUSTOMER_ORDER_ISSUED_FLAG', 'FL_IS_ONLINE_SALES_FLAG', 'FL_IS_DIRECT_SALES_FLAG', 'FL_IS_SALES_INVOICE_ISSUED_FLAG', 'FL_IS_PROPERTY_STOCK_FLAG', 'FL_IS_PROPERTY_STOCK_IN_TRANSIT_FLAG', 'FL_IS_PROPERTY_STOCK_ARRIVED_TO_SALES_MARKET_FLAG', 'FL_IS_NETWORK_STOCK_FLAG', 'FL_IS_NETWORK_STOCK_IN_TRANSIT_FLAG', 'FL_IS_VEHICLE_NOT_SOLD_WITH_SALES_INVOICE_FLAG', 'FL_IS_FACTORY_PRODUCTION_PLANNED_FLAG', 'FL_IS_VEHICLE_PASSED_TO_SALES_FLAG', 'FL_IS_VEHICLE_ORDER_IN_PLANT_FLAG', 'FL_IS_SHIPPED_TO_COMPOUND_FLAG', 'FL_IS_VEHICLE_ORDER_IN_COUNTRY_FLAG', 'FL_IS_CUSTOMS_OFFICE_OUTBOUND_FLAG', 'FL_

## 5. Processing Categorical Code Columns (CD_)
This section handles columns prefixed with `CD_`, representing categorical codes.
The strategy involves:
* Dropping columns that are entirely null.
* Applying different imputation strategies based on the percentage of missing values:
    * 0-5% missing: Impute with mode.
    * 5-35% missing: Impute with a constant string "UNK" (Unknown).
    * >35% missing: Drop column or apply specific manual imputation.
* Creating flag columns to indicate where imputation occurred for mode/"UNK" filled columns.

In [36]:
# Identify columns starting with "CD_"
code_cols_initial = filtered_df.columns[
    filtered_df.columns.str.startswith("CD_")
].tolist()  #
print(f"Found {len(code_cols_initial)} columns starting with 'CD_'.")

# Drop CD_ columns that are 100% null
cols_to_drop_all_null = [
    col for col in code_cols_initial if filtered_df[col].isna().all()
]  #
if cols_to_drop_all_null:
    filtered_df.drop(columns=cols_to_drop_all_null, inplace=True)
    print(f"Dropped CD_ columns with all null values: {cols_to_drop_all_null}")

# Update the list of CD_ columns after dropping all-null ones
code_cols_to_process = filtered_df.columns[
    filtered_df.columns.str.startswith("CD_")
].tolist()  #

# Calculate the percentage of missing values for the remaining CD_ columns
na_percentage_cd = (
    filtered_df[code_cols_to_process].isna().mean() * 100
    if code_cols_to_process
    else pd.Series(dtype=float)
)  #


# Helper function for imputation and adding a missing flag
def impute_with_flag(df, column_name, fill_value_or_func):
    """Imputes missing values and adds a flag column."""
    flag_col_name = f"is_missing_{column_name}"
    df[flag_col_name] = df[column_name].isna().astype(int)  #

    fill_value = fill_value_or_func
    if callable(fill_value_or_func):
        # Check if series is all NaN before calling mode, to avoid empty mode result
        if df[column_name].notna().any():
            fill_value = fill_value_or_func(df, column_name)
        else:  # If all NaN, mode is empty, fill with a default like "UNK"
            fill_value = "UNK_ALL_NAN"
            print(
                f"Warning: Column {column_name} is all NaN. Imputing with '{fill_value}'."
            )

    df[column_name].fillna(fill_value, inplace=True)  #
    print(
        f"Imputed '{column_name}' with '{fill_value}' (or mode) and created flag '{flag_col_name}'."
    )
    return df


# Columns with 0% < NA < 5%: Impute with mode
cols_impute_mode = na_percentage_cd[
    (na_percentage_cd > 0) & (na_percentage_cd < 5)
].index.tolist()  #
for col in cols_impute_mode:
    filtered_df = impute_with_flag(
        filtered_df,
        col,
        lambda df_lambda, c: df_lambda[c].mode()[0]
        if not df_lambda[c].mode().empty
        else "UNK_MODE_EMPTY",
    )

# Columns with 5% <= NA < 20%: Impute with "UNK"
cols_impute_unk_medium = na_percentage_cd[
    (na_percentage_cd >= 5) & (na_percentage_cd < 20)
].index.tolist()  #
for col in cols_impute_unk_medium:
    filtered_df = impute_with_flag(filtered_df, col, "UNK")

# Columns with 20% <= NA < 35%: Impute with "UNK"
cols_impute_unk_high = na_percentage_cd[
    (na_percentage_cd >= 20) & (na_percentage_cd < 35)
].index.tolist()  #
for col in cols_impute_unk_high:
    filtered_df = impute_with_flag(filtered_df, col, "UNK")

# Columns with NA >= 35%: Drop these columns as per original script's specific list
cols_to_drop_high_na_specific = [
    "CD_VEHICLE_ORDER_CODE",
    "CD_SHIPPING_ORDER_TO_DEALER_CODE",
    "CD_SOURCE_SYSTEM_BLOCK_REASON_TYPE",
    "CD_SOURCE_SYSTEM_BLOCK_REASON_CODE",
    "CD_GLOBAL_FINAL_CUSTOMER_ORDER_CODE",
]  #
actual_cols_to_drop_high_na = [
    col for col in cols_to_drop_high_na_specific if col in filtered_df.columns
]
if actual_cols_to_drop_high_na:
    filtered_df.drop(columns=actual_cols_to_drop_high_na, inplace=True)
    print(
        f"Dropped CD_ columns with high NA (>35% or specifically listed): {actual_cols_to_drop_high_na}"
    )

# Specific manual imputation for certain CD_ columns (example from original script)
if "CD_NETWORK_SUBJECT_ORDER_CODE" in filtered_df.columns:
    if filtered_df["CD_NETWORK_SUBJECT_ORDER_CODE"].isna().any():
        filtered_df["CD_NETWORK_SUBJECT_ORDER_CODE"].fillna(
            "NOT_DEALER_ORDER", inplace=True
        )  #
        print("Filled NaNs in 'CD_NETWORK_SUBJECT_ORDER_CODE' with 'NOT_DEALER_ORDER'.")

subchannel_col = "CD_OPERATIONAL_PLANNING_SALES_SUBCHANNEL_ORIGINAL_CODE"
if subchannel_col in filtered_df.columns:
    if filtered_df[subchannel_col].isna().any():
        # This column was listed with >35% NA in the original script, but then manually handled.
        # Re-applying the specific logic from the script if it wasn't dropped above.
        filtered_df[f"is_missing_{subchannel_col}"] = (
            filtered_df[subchannel_col].isna().astype(int)
        )  #
        filtered_df[subchannel_col].fillna("UNK", inplace=True)  #
        print(f"Filled NaNs in '{subchannel_col}' with 'UNK' and created flag.")

print("\nCD_ columns processed. Check for remaining NaNs:")
final_cd_cols = filtered_df.columns[filtered_df.columns.str.startswith("CD_")]
if not final_cd_cols.empty:
    print(filtered_df[final_cd_cols].isna().sum().loc[lambda x: x > 0])
else:
    print("No CD_ columns remaining or all are handled.")

Found 46 columns starting with 'CD_'.
Dropped CD_ columns with all null values: ['CD_VEHICLE_ORDER_CARFLOW_STATUS_CODE', 'CD_SOURCE_SYSTEM_VEHICLE_SHOWROOM_STATUS_CODE']
Imputed 'CD_NETWORK_SUBJECT_CODE' with 'GBW1350' (or mode) and created flag 'is_missing_CD_NETWORK_SUBJECT_CODE'.
Imputed 'CD_FINAL_CUSTOMER_ORDER_CODE' with '220000233' (or mode) and created flag 'is_missing_CD_FINAL_CUSTOMER_ORDER_CODE'.
Imputed 'CD_NETWORK_SUBJECT_FOR_INVOICING_CODE' with 'GBW1350' (or mode) and created flag 'is_missing_CD_NETWORK_SUBJECT_FOR_INVOICING_CODE'.
Imputed 'CD_SALES_REGION_CODE' with 'UNK' (or mode) and created flag 'is_missing_CD_SALES_REGION_CODE'.
Imputed 'CD_SHIPPING_ORDER_TO_NSC_COMPOUND_CODE' with 'UNK' (or mode) and created flag 'is_missing_CD_SHIPPING_ORDER_TO_NSC_COMPOUND_CODE'.
Imputed 'CD_VEHICLE_ORDER_ORIGIN_CODE' with 'UNK' (or mode) and created flag 'is_missing_CD_VEHICLE_ORDER_ORIGIN_CODE'.
Imputed 'CD_VEHICLE_LAST_LOCATION_CODE' with 'UNK' (or mode) and created flag 'is_mi

## 6. Processing General Text Columns (GN_)
This section handles columns prefixed with `GN_`, which are general text or name columns.
Steps include:
* Dropping columns that are entirely null.
* Dropping columns containing 'GUID' in their name (unique identifiers, usually not useful as features).
* Dropping a specific list of GN_ columns deemed unnecessary or having too many missing values.
* Imputing `GN_BRAND_ORIGINAL_NAME` using a mapping from `CD_BRAND_CODE`.
* Imputing `GN_NETWORK_SUBJECT_NAME` with "UNK".
* Imputing `GN_VEHICLE_CAR_LINE_NAME` with its mode (conditional on `CD_BRAND_CODE` == "C" in original script, generalized here to overall mode if specific mode fails).
* Converting remaining GN_ columns (that were object type) to string type for consistency.

In [37]:
# Select initial GN_ columns
gn_cols_initial = filtered_df.columns[
    filtered_df.columns.str.startswith("GN_")
].tolist()  #
print(f"Found {len(gn_cols_initial)} columns starting with 'GN_'.")

# Drop GN_ columns that are 100% empty (all NaN)
empty_gn_cols = [col for col in gn_cols_initial if filtered_df[col].isna().all()]  #
if empty_gn_cols:
    filtered_df.drop(columns=empty_gn_cols, inplace=True)
    print(f"Dropped GN_ columns with all null values: {empty_gn_cols}")

# Drop GN_ columns containing 'GUID' in their name
guid_gn_cols = filtered_df.columns[
    filtered_df.columns.str.contains("GUID") & filtered_df.columns.str.startswith("GN_")
].tolist()  #
if guid_gn_cols:
    filtered_df.drop(columns=guid_gn_cols, inplace=True)
    print(f"Dropped GN_ columns containing 'GUID': {guid_gn_cols}")

# Update the list of GN_ columns to process
gn_cols_to_process = filtered_df.columns[
    filtered_df.columns.str.startswith("GN_")
].tolist()

# Display NA count for remaining GN_ columns (informational, as in original script)
if gn_cols_to_process:
    print(
        "\nNA percentage in remaining GN_ columns before further dropping/imputation:"
    )
    na_counts_gn = filtered_df[gn_cols_to_process].isna().mean() * 100  #
    print(na_counts_gn[na_counts_gn > 0])
else:
    print("No GN_ columns remaining after initial drops.")

Found 36 columns starting with 'GN_'.
Dropped GN_ columns with all null values: ['GN_VEHICLE_RENT_A_CAR_FLEET_DOCUMENTS_DISPATCH_TRACKING_CODE']
Dropped GN_ columns containing 'GUID': ['GN_SYSTEM_ORDER_GUID', 'GN_OPERATIONAL_PLANNING_SALES_CHANNEL_GUID', 'GN_OPERATIONAL_PLANNING_SALES_SUBCHANNEL_GUID', 'GN_NETWORK_SUBJECT_FOR_INVOICING_GUID', 'GN_VEHICLE_ORDER_GUID', 'GN_FINAL_CUSTOMER_ORDER_GUID', 'GN_DESTINATION_LOGISTIC_COMPOUND_GUID', 'GN_DESTINATION_LOGISTIC_COMPOUND_TYPE_GUID', 'GN_NETWORK_SUBJECT_ORDER_GUID', 'GN_CUSTOMER_SALES_CHANNEL_GUID', 'GN_CUSTOMER_SALES_SUBCHANNEL_GUID', 'GN_DISTRIBUTION_SALES_CHANNEL_GUID', 'GN_DISTRIBUTION_SALES_SUBCHANNEL_GUID']

NA percentage in remaining GN_ columns before further dropping/imputation:
GN_BRAND_ORIGINAL_NAME                                    54.920803
GN_OPERATIONAL_PLANNING_SALES_CHANNEL_ORIGINAL_NAME       23.072862
GN_OPERATIONAL_PLANNING_SALES_CHANNEL_NAME                58.152059
GN_OPERATIONAL_PLANNING_SALES_SUBCHANNEL_ORIGINA

In [38]:
# Specific list of GN_ columns to drop (based on original script's decision, likely due to high NAs or low relevance)
gn_cols_to_drop_specific = [
    "GN_OPERATIONAL_PLANNING_SALES_CHANNEL_ORIGINAL_NAME",
    "GN_OPERATIONAL_PLANNING_SALES_CHANNEL_NAME",
    "GN_OPERATIONAL_PLANNING_SALES_SUBCHANNEL_ORIGINAL_NAME",
    "GN_SALES_INVOICE_NUMBER",
    "GN_VEHICLE_ORDER_SECTION_NUMBER",
    "GN_INCOTERM_OUTBOUND_FLOW",
    "GN_VEHICLE_ORDER_STATUS_DETAIL_NAME",
    "GN_VEHICLE_LAST_LOCATION_NAME",
    "GN_SHIPPER_DEALER_NAME",
    "GN_SHIPPING_ZONE_NAME",
    "GN_DESTINATION_LOGISTIC_COMPOUND_NAME",
    "GN_SOURCE_SYSTEM_BLOCK_REASON_NAME",
    "GN_CUSTOMER_SALES_CHANNEL_ORIGINAL_NAME",
    # 'GN_CUSTOMER_SALES_CHANNEL_NAME', # This was kept in some versions, verify necessity
    "GN_DISTRIBUTION_SALES_CHANNEL_ORIGINAL_NAME",
    "GN_DISTRIBUTION_SALES_SUBCHANNEL_ORIGINAL_NAME",
    "GN_CUSTOMER_SALES_SUBCHANNEL_ORIGINAL_NAME",
]  #

actual_gn_cols_to_drop = [
    col for col in gn_cols_to_drop_specific if col in filtered_df.columns
]
if actual_gn_cols_to_drop:
    filtered_df.drop(columns=actual_gn_cols_to_drop, inplace=True)  #
    print(f"Dropped specifically listed GN_ columns: {actual_gn_cols_to_drop}")

# Update gn_cols_to_process again
gn_cols_to_process = filtered_df.columns[
    filtered_df.columns.str.startswith("GN_")
].tolist()

Dropped specifically listed GN_ columns: ['GN_OPERATIONAL_PLANNING_SALES_CHANNEL_ORIGINAL_NAME', 'GN_OPERATIONAL_PLANNING_SALES_CHANNEL_NAME', 'GN_OPERATIONAL_PLANNING_SALES_SUBCHANNEL_ORIGINAL_NAME', 'GN_SALES_INVOICE_NUMBER', 'GN_VEHICLE_ORDER_SECTION_NUMBER', 'GN_INCOTERM_OUTBOUND_FLOW', 'GN_VEHICLE_ORDER_STATUS_DETAIL_NAME', 'GN_VEHICLE_LAST_LOCATION_NAME', 'GN_SHIPPER_DEALER_NAME', 'GN_SHIPPING_ZONE_NAME', 'GN_DESTINATION_LOGISTIC_COMPOUND_NAME', 'GN_SOURCE_SYSTEM_BLOCK_REASON_NAME', 'GN_CUSTOMER_SALES_CHANNEL_ORIGINAL_NAME', 'GN_DISTRIBUTION_SALES_CHANNEL_ORIGINAL_NAME', 'GN_DISTRIBUTION_SALES_SUBCHANNEL_ORIGINAL_NAME', 'GN_CUSTOMER_SALES_SUBCHANNEL_ORIGINAL_NAME']


In [39]:
# Define mapping for GN_BRAND_ORIGINAL_NAME imputation from CD_BRAND_CODE
brand_code_to_name_map = {
    "B": "Abarth",
    "A": "Alfa Romeo",
    "L": "Chrysler",
    "W": "Dodge",
    "I": "Fiat",
    "J": "Jeep",
    "E": "Lancia",
    "O": "RAM",
    "X": "Maserati",
    "C": "Citroën",
    "P": "Peugeot",
    "S": "DS",
    "G": "Opel",
}  #

In [40]:
# Impute 'GN_BRAND_ORIGINAL_NAME' where it's missing, using the mapping
brand_name_col = "GN_BRAND_ORIGINAL_NAME"
brand_code_col = "CD_BRAND_CODE"
if brand_name_col in filtered_df.columns and brand_code_col in filtered_df.columns:
    if filtered_df[brand_name_col].isna().any():
        # Create a series of mapped names from the brand code
        mapped_brand_names = filtered_df[brand_code_col].map(brand_code_to_name_map)  #
        # Fill NaNs in GN_BRAND_ORIGINAL_NAME with these mapped names
        filtered_df[brand_name_col] = filtered_df[brand_name_col].fillna(
            mapped_brand_names
        )  #
        # For any remaining NaNs (if CD_BRAND_CODE was NaN or not in map), fill with "Unknown_Brand"
        if filtered_df[brand_name_col].isna().any():
            filtered_df[brand_name_col].fillna("Unknown_Brand", inplace=True)
        print(
            f"Imputed missing values in '{brand_name_col}' using '{brand_code_col}' mapping and 'Unknown_Brand' as fallback."
        )
else:
    print(
        f"Skipping imputation for '{brand_name_col}' as it or '{brand_code_col}' is not in DataFrame."
    )

Imputed missing values in 'GN_BRAND_ORIGINAL_NAME' using 'CD_BRAND_CODE' mapping and 'Unknown_Brand' as fallback.


In [41]:
# Impute 'GN_NETWORK_SUBJECT_NAME' with "UNK" and add a flag (using the helper function)
network_subject_col = "GN_NETWORK_SUBJECT_NAME"
if (
    network_subject_col in filtered_df.columns
    and filtered_df[network_subject_col].isna().any()
):
    filtered_df = impute_with_flag(filtered_df, network_subject_col, "UNK")  #
else:
    if network_subject_col not in filtered_df.columns:
        print(f"Column '{network_subject_col}' not found for imputation.")
    else:
        print(f"No missing values in '{network_subject_col}' to impute.")

Imputed 'GN_NETWORK_SUBJECT_NAME' with 'UNK' (or mode) and created flag 'is_missing_GN_NETWORK_SUBJECT_NAME'.


In [42]:
# Impute 'GN_VEHICLE_CAR_LINE_NAME'
# Original script had specific logic: df[df["CD_BRAND_CODE"] == "C"]["GN_VEHICLE_CAR_LINE_NAME"].mode().to_list()[0]
# This implies filling NaNs with the mode of car line names for Citroën ("C") vehicles.
# We'll generalize: if 'C' exists and has a mode, use it. Otherwise, use overall mode, then "Unknown_Car_Line".
car_line_col = "GN_VEHICLE_CAR_LINE_NAME"
if car_line_col in filtered_df.columns and filtered_df[car_line_col].isna().any():
    fill_value_car_line = "Unknown_Car_Line"  # Default fallback
    if (
        brand_code_col in filtered_df.columns
        and "C" in filtered_df[brand_code_col].unique()
    ):
        mode_for_C = filtered_df[filtered_df[brand_code_col] == "C"][
            car_line_col
        ].mode()
        if not mode_for_C.empty:
            fill_value_car_line = mode_for_C[0]
            print(
                f"Using mode for brand 'C' for '{car_line_col}': {fill_value_car_line}"
            )
        else:
            # Fallback to overall mode if brand 'C' has no mode for car line
            overall_mode = filtered_df[car_line_col].mode()
            if not overall_mode.empty:
                fill_value_car_line = overall_mode[0]
                print(f"Using overall mode for '{car_line_col}': {fill_value_car_line}")
    else:
        # Fallback to overall mode if brand 'C' not present or CD_BRAND_CODE missing
        overall_mode = filtered_df[car_line_col].mode()
        if not overall_mode.empty:
            fill_value_car_line = overall_mode[0]
            print(f"Using overall mode for '{car_line_col}': {fill_value_car_line}")

    filtered_df[car_line_col].fillna(fill_value_car_line, inplace=True)  #
    print(f"Filled NaNs in '{car_line_col}' with '{fill_value_car_line}'.")
else:
    if car_line_col not in filtered_df.columns:
        print(f"Column '{car_line_col}' not found for imputation.")
    else:
        print(f"No missing values in '{car_line_col}' to impute.")

No missing values in 'GN_VEHICLE_CAR_LINE_NAME' to impute.


In [43]:
# Convert remaining GN_ columns (that were object and are now processed) to string type
# This ensures consistency and avoids issues with mixed types if any imputation resulted in numbers/bools etc.
final_gn_cols_to_convert = [
    "GN_BRAND_ORIGINAL_NAME",
    "GN_VEHICLE_COMPLETE_DESCRIPTION_NAME",
    "GN_VEHICLE_CAR_LINE_NAME",
    "GN_NETWORK_SUBJECT_NAME",
    # Add any other GN_ columns that should be string and were kept
]  #

for col in final_gn_cols_to_convert:
    if col in filtered_df.columns:
        # Fill any potential post-imputation NaNs (e.g. if a mode was NaN) with a placeholder before converting to string
        if filtered_df[col].isna().any():
            filtered_df[col].fillna("UNKNOWN_GN_VALUE", inplace=True)
        filtered_df[col] = filtered_df[col].astype("string")  #

print(f"Converted specified GN_ columns to 'string' type.")
gn_cols_final_check = filtered_df.columns[filtered_df.columns.str.startswith("GN_")]
if not gn_cols_final_check.empty:
    print("\nData types of GN_ columns after processing:")
    print(filtered_df[gn_cols_final_check].dtypes)  #
    print("\nMissing values in GN_ columns after processing:")
    print(filtered_df[gn_cols_final_check].isna().sum().loc[lambda x: x > 0])
else:
    print("No GN_ columns remaining.")

Converted specified GN_ columns to 'string' type.

Data types of GN_ columns after processing:
GN_BRAND_ORIGINAL_NAME                  string[python]
GN_VEHICLE_COMPLETE_DESCRIPTION_NAME    string[python]
GN_VEHICLE_CAR_LINE_NAME                string[python]
GN_NETWORK_SUBJECT_NAME                 string[python]
GN_VEHICLE_ORDER_FLOW_TYPE                       int64
GN_CUSTOMER_SALES_CHANNEL_NAME                  object
dtype: object

Missing values in GN_ columns after processing:
GN_CUSTOMER_SALES_CHANNEL_NAME    706
dtype: int64


## 7. Processing Numerical Columns (NM_)
This section handles columns prefixed with `NM_`, which are numerical.
The original script drops all `NM_` columns. This step is retained here.

In [44]:
# Identify columns starting with "NM_"
nm_cols = filtered_df.columns[filtered_df.columns.str.startswith("NM_")].tolist()  #

if nm_cols:
    filtered_df.drop(columns=nm_cols, inplace=True)  #
    print(f"Dropped all NM_ columns: {nm_cols}")
else:
    print("No NM_ columns found to drop.")

Dropped all NM_ columns: ['NM_VEHICLE_ORDER_OEM_STOCK_AGEING_NUMBER', 'NM_VEHICLE_ORDER_DEALER_STOCK_AGEING_NUMBER', 'NM_NEW_VEHICLE_ORDER_GENERAL_DISTRIBUTOR_STOCK_AGEING_NUMBER']


## 8. Feature Engineering
This section creates new features from existing columns to potentially improve model performance or provide richer insights.
Features created include:
* `Is_Delayed`: Binary flag indicating if the delivery was delayed.
* `REG_Delay_Days`: Delay in days, clipped at 0 (negative delays treated as 0).
* Various time differences between key process stages (e.g., `ORDER_TO_PRODUCTION_DAYS`).
* `CUSTOMS_CLEARANCE_DAYS`: Duration of customs clearance (NaN for domestic).
* `TOTAL_LEAD_VS_EXPECTED_DAYS`: Difference between actual and expected total lead time.
* Flags for missing key dates (redundant if imputation flags already exist, but kept per original script).
* Date components like `ORDER_DAY_OF_WEEK` and `PRODUCTION_MONTH`.

In [45]:
# Create 'Is_Delayed' flag (binary: 1 if delayed, 0 otherwise)
# 'Delay_days' is a Timedelta object, so we access .dt.days
if "Delay_days" in filtered_df.columns:
    filtered_df["Is_Delayed"] = (filtered_df["Delay_days"].dt.days > 0).astype(int)  #
    print("Created 'Is_Delayed' feature.")
else:
    print(
        "Warning: 'Delay_days' column not found. 'Is_Delayed' feature cannot be created."
    )

Created 'Is_Delayed' feature.


In [46]:
# Create 'REG_Delay_Days' - delay in days, with negative values clipped to 0 (no "early" credit)
if "Delay_days" in filtered_df.columns:
    filtered_df["REG_Delay_Days"] = filtered_df["Delay_days"].dt.days.clip(lower=0)  #
    print("Created 'REG_Delay_Days' feature.")

    # Drop the original 'Delay_days' (Timedelta object) as its information is now in 'Is_Delayed' and 'REG_Delay_Days'
    filtered_df.drop(columns=["Delay_days"], inplace=True)  #
    print("Dropped original 'Delay_days' column.")
else:
    print(
        "Warning: 'Delay_days' column not found. 'REG_Delay_Days' feature cannot be created and 'Delay_days' cannot be dropped."
    )

Created 'REG_Delay_Days' feature.
Dropped original 'Delay_days' column.


In [47]:
# Time from commercial order to factory production
# if "DT_VEHICLE_FACTORY_PRODUCTION_DATE" in filtered_df.columns and \
#   "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE" in filtered_df.columns:
#    filtered_df["ORDER_TO_PRODUCTION_DAYS"] = (
#        filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"]
#        - filtered_df["DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"]
#    ).dt.days #
#    print("Created 'ORDER_TO_PRODUCTION_DAYS' feature.")
# else:
#    print("Warning: Required date columns missing for 'ORDER_TO_PRODUCTION_DAYS'.")

In [48]:
# Time from factory production to ready to ship from logistic plant
# if "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE" in filtered_df.columns and \
#   "DT_VEHICLE_FACTORY_PRODUCTION_DATE" in filtered_df.columns:
#    filtered_df["PRODUCTION_TO_SHIPPING_READY_DAYS"] = (
#        filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"]
#        - filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"]
#    ).dt.days #
#    print("Created 'PRODUCTION_TO_SHIPPING_READY_DAYS' feature.")
# else:
#    print("Warning: Required date columns missing for 'PRODUCTION_TO_SHIPPING_READY_DAYS'.")

In [49]:
# Time from ready to ship to shipping order creation (commented out in original script, retained as such)
# if "DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE" in filtered_df.columns and \
#    "DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE" in filtered_df.columns:
#     filtered_df["SHIPPING_READY_TO_SHIPPING_ORDER_DAYS"] = (
#         filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"]
#         - filtered_df["DT_READY_TO_SHIP_FROM_LOGISTIC_PLANT_COMPOUND_DATE"]
#     ).dt.days #
#     print("Created 'SHIPPING_READY_TO_SHIPPING_ORDER_DAYS' feature (if uncommented).")
# else:
#     print("Warning: Required date columns missing for 'SHIPPING_READY_TO_SHIPPING_ORDER_DAYS' (if uncommented).")

In [50]:
# Time from shipping order creation to arrival at destination by transporter
# if "DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE" in filtered_df.columns and \
#   "DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE" in filtered_df.columns:
#    filtered_df["SHIPPING_TRANSIT_DAYS"] = (
#        filtered_df["DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE"]
#        - filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"]
#    ).dt.days #
#    print("Created 'SHIPPING_TRANSIT_DAYS' feature.")
# else:
#    print("Warning: Required date columns missing for 'SHIPPING_TRANSIT_DAYS'.")

In [51]:
# Customs clearance time
# if "DT_CUSTOMS_OFFICE_OUTBOUND_DATE" in filtered_df.columns and \
#   "DT_CUSTOMS_OFFICE_INBOUND_DATE" in filtered_df.columns and \
#   "IS_INTERNATIONAL" in filtered_df.columns:
#    filtered_df["CUSTOMS_CLEARANCE_DAYS"] = (
#        filtered_df["DT_CUSTOMS_OFFICE_OUTBOUND_DATE"]
#        - filtered_df["DT_CUSTOMS_OFFICE_INBOUND_DATE"]
#    ).dt.days #

#    # Set to NaN (or a specific value like -1 or 0) for domestic shipments where customs dates are not applicable
#    filtered_df.loc[filtered_df["IS_INTERNATIONAL"] == 0, "CUSTOMS_CLEARANCE_DAYS"] = np.nan #
#    print("Created 'CUSTOMS_CLEARANCE_DAYS' feature (NaN for domestic).")
# else:
#    print("Warning: Required columns missing for 'CUSTOMS_CLEARANCE_DAYS'.")

In [52]:
# Difference between actual total lead time and expected total lead time
# if "DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE" in filtered_df.columns and \
#   "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE" in filtered_df.columns and \
#   "DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE" in filtered_df.columns:
#    actual_lead_time_days = (
#        filtered_df["DT_ARRIVAL_AT_DESTINATION_BY_TRANSPORTER_DATE"]
#        - filtered_df["DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"]
#    ).dt.days #
#
#    expected_lead_time_days = (
#        filtered_df["DT_EXPECTED_DELIVERY_TO_FINAL_CUSTOMER_DATE"]
#        - filtered_df["DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"]
#    ).dt.days #
#
#    filtered_df["TOTAL_LEAD_VS_EXPECTED_DAYS"] = actual_lead_time_days - expected_lead_time_days #
#    print("Created 'TOTAL_LEAD_VS_EXPECTED_DAYS' feature.")
# else:
#    print("Warning: Required columns missing for 'TOTAL_LEAD_VS_EXPECTED_DAYS'.")

In [53]:
# Flag for missing shipping order date (This was already created as 'MISSING_SHIPPING_ORDER_FLAG' during imputation)
# Re-creating it as per original script logic (might be redundant if previous flags are kept)
# if "DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE" in filtered_df.columns:
#    filtered_df["MISSING_SHIPPING_ORDER_DATE_FE_FLAG"] = (
#        filtered_df["DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE"].isna().astype(int)
#    ) #
# Note: If imputation was successful, this should be all zeros. If NaNs persist, it means imputation failed or wasn't applicable.
#    print("Created/updated 'MISSING_SHIPPING_ORDER_DATE_FE_FLAG'.")
# else:
#    print("Warning: 'DT_SHIPPING_ORDER_TO_NSC_COMPOUND_CREATION_DATE' not found for flag creation.")

In [54]:
# Flag for missing customs inbound date
# if "DT_CUSTOMS_OFFICE_INBOUND_DATE" in filtered_df.columns:
#    filtered_df["MISSING_CUSTOMS_INBOUND_DATE_FE_FLAG"] = (
#        filtered_df["DT_CUSTOMS_OFFICE_INBOUND_DATE"].isna().astype(int)
#    ) #
#    # This flag will be 1 for domestic shipments or where data was truly missing.
#    print("Created 'MISSING_CUSTOMS_INBOUND_DATE_FE_FLAG'.")
# else:
#    print("Warning: 'DT_CUSTOMS_OFFICE_INBOUND_DATE' not found for flag creation.")

In [55]:
# Day of the week for the commercial order date (Monday=0, Sunday=6)
if "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE" in filtered_df.columns:
    filtered_df["ORDER_DAY_OF_WEEK"] = filtered_df[
        "DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE"
    ].dt.dayofweek  #
    print("Created 'ORDER_DAY_OF_WEEK' feature.")
else:
    print(
        "Warning: 'DT_COMMERCIAL_ORDER_FIRST_ENTRY_DATE' not found for 'ORDER_DAY_OF_WEEK'."
    )

Created 'ORDER_DAY_OF_WEEK' feature.


In [56]:
# Month of the vehicle factory production date
# if "DT_VEHICLE_FACTORY_PRODUCTION_DATE" in filtered_df.columns:
# Ensure the column is datetime before accessing .dt accessor
#    if pd.api.types.is_datetime64_any_dtype(filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"]):
#        filtered_df["PRODUCTION_MONTH"] = (
#            filtered_df["DT_VEHICLE_FACTORY_PRODUCTION_DATE"].dt.month
#        ) #
#        print("Created 'PRODUCTION_MONTH' feature.")
#    else:
#        print("Warning: 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' is not datetime. Cannot create 'PRODUCTION_MONTH'.")
# else:
#    print("Warning: 'DT_VEHICLE_FACTORY_PRODUCTION_DATE' not found for 'PRODUCTION_MONTH'.")

## 9. Final Data Cleaning and Validation
This section performs a final check for missing values and applies specific handling for remaining NaNs in certain columns based on the original script's logic.

In [57]:
# Identify columns with any remaining missing values
remaining_na_summary = filtered_df.isna().sum()  #
columns_with_remaining_na = remaining_na_summary[remaining_na_summary > 0].to_dict()  #

if columns_with_remaining_na:
    print("Columns with remaining missing values before final handling:")
    for col, count in columns_with_remaining_na.items():
        print(f"- {col}: {count} missing values")
else:
    print(
        "No missing values remaining in the DataFrame before final specific handling."
    )

Columns with remaining missing values before final handling:
- DT_CUSTOMS_OFFICE_INBOUND_DATE: 7109 missing values
- DT_CUSTOMS_OFFICE_OUTBOUND_DATE: 7108 missing values
- CD_DESTINATION_LOGISTIC_COMPOUND_CODE: 5189 missing values
- CD_DESTINATION_LOGISTIC_COMPOUND_TYPE_CODE: 5189 missing values
- CD_SOURCE_SYSTEM_VEHICLE_DEMONSTRATION_STATUS_CODE: 4654 missing values
- GN_CUSTOMER_SALES_CHANNEL_NAME: 706 missing values
- CD_DISTRIBUTION_SALES_SUBCHANNEL_ORIGINAL_CODE: 4922 missing values


In [58]:
# Handle 'CD_DESTINATION_LOGISTIC_COMPOUND_CODE'
compound_code_col = "CD_DESTINATION_LOGISTIC_COMPOUND_CODE"
if compound_code_col in filtered_df.columns:
    # Group rare codes (less than 1% frequency) into "Other"
    if (
        filtered_df[compound_code_col].notna().any()
    ):  # Ensure there are non-NA values to calculate frequencies
        compound_code_counts = filtered_df[compound_code_col].value_counts(
            normalize=True
        )  #
        rare_codes = compound_code_counts[compound_code_counts < 0.01].index.tolist()  #
        if rare_codes:
            filtered_df[compound_code_col] = filtered_df[compound_code_col].replace(
                rare_codes, "Other"
            )  #
            print(f"Grouped rare codes in '{compound_code_col}' into 'Other'.")

    # Impute remaining missing values (NaNs) as "Unknown"
    if filtered_df[compound_code_col].isna().any():
        filtered_df[compound_code_col].fillna("Unknown", inplace=True)  #
        print(f"Filled NaNs in '{compound_code_col}' with 'Unknown'.")
else:
    print(f"Column '{compound_code_col}' not found for final handling.")

Grouped rare codes in 'CD_DESTINATION_LOGISTIC_COMPOUND_CODE' into 'Other'.
Filled NaNs in 'CD_DESTINATION_LOGISTIC_COMPOUND_CODE' with 'Unknown'.


In [59]:
# Handle 'CD_DESTINATION_LOGISTIC_COMPOUND_TYPE_CODE' and create 'Destination_Type'
compound_type_col = "CD_DESTINATION_LOGISTIC_COMPOUND_TYPE_CODE"
if compound_type_col in filtered_df.columns:
    # Impute missing type codes with "Unknown"
    if filtered_df[compound_type_col].isna().any():
        filtered_df[compound_type_col].fillna("Unknown", inplace=True)  #
        print(f"Filled NaNs in '{compound_type_col}' with 'Unknown'.")

    # Combine compound code and type into a new feature 'Destination_Type'
    if compound_code_col in filtered_df.columns:  # Ensure the code column also exists
        filtered_df["Destination_Type"] = (
            filtered_df[compound_code_col].astype(str)
            + "_"
            + filtered_df[compound_type_col].astype(str)
        )  #
        print("Created 'Destination_Type' feature by combining compound code and type.")
    else:
        print(f"Cannot create 'Destination_Type' as '{compound_code_col}' is missing.")
else:
    print(f"Column '{compound_type_col}' not found for final handling.")

Filled NaNs in 'CD_DESTINATION_LOGISTIC_COMPOUND_TYPE_CODE' with 'Unknown'.
Created 'Destination_Type' feature by combining compound code and type.


In [60]:
# Handle 'CD_SOURCE_SYSTEM_VEHICLE_DEMONSTRATION_STATUS_CODE' by creating 'IS_DEMONSTRATION_VEHICLE' flag
demo_status_code_col = "CD_SOURCE_SYSTEM_VEHICLE_DEMONSTRATION_STATUS_CODE"
if demo_status_code_col in filtered_df.columns:
    # If code exists (not NaN), it's a demonstration vehicle (1), otherwise not (0).
    filtered_df["IS_DEMONSTRATION_VEHICLE"] = (
        filtered_df[demo_status_code_col].notna().astype(int)
    )  #
    # The original script also fills NaNs in the new flag with 0, which notna().astype(int) already does implicitly.
    # filtered_df["IS_DEMONSTRATION_VEHICLE"].fillna(0, inplace=True) # Redundant but harmless
    print("Created 'IS_DEMONSTRATION_VEHICLE' flag.")

    # Drop the original demonstration status code column
    filtered_df.drop(columns=[demo_status_code_col], inplace=True)  #
    print(f"Dropped original column '{demo_status_code_col}'.")
else:
    print(
        f"Column '{demo_status_code_col}' not found for creating demonstration vehicle flag."
    )

Created 'IS_DEMONSTRATION_VEHICLE' flag.
Dropped original column 'CD_SOURCE_SYSTEM_VEHICLE_DEMONSTRATION_STATUS_CODE'.


In [61]:
dist_subchannel_col = "CD_DISTRIBUTION_SALES_SUBCHANNEL_ORIGINAL_CODE"

if dist_subchannel_col in filtered_df.columns:
    if filtered_df[dist_subchannel_col].isna().any():
        # Convertir la colonne en type chaîne de caractères avant de remplir les NaN
        filtered_df[dist_subchannel_col] = filtered_df[dist_subchannel_col].astype(str)
        # Remplacer les NaN par 'Unknown'
        filtered_df[dist_subchannel_col] = filtered_df[dist_subchannel_col].fillna(
            "Unknown"
        )
        print(f"Remplacé les NaN dans '{dist_subchannel_col}' par 'Unknown'.")
    else:
        # S'assurer que la colonne est de type chaîne de caractères
        filtered_df[dist_subchannel_col] = filtered_df[dist_subchannel_col].astype(str)
        print(
            f"Assuré que '{dist_subchannel_col}' est de type chaîne de caractères pour l'export Parquet."
        )
else:
    print(f"Colonne '{dist_subchannel_col}' non trouvée pour le traitement final.")


Remplacé les NaN dans 'CD_DISTRIBUTION_SALES_SUBCHANNEL_ORIGINAL_CODE' par 'Unknown'.


In [62]:
# Final check for any remaining missing values after all processing
final_na_summary = filtered_df.isna().sum()
final_columns_with_na = final_na_summary[final_na_summary > 0].to_dict()  #

if final_columns_with_na:
    print("\n--- FINAL REPORT: Columns with remaining missing values ---")
    for col, count in final_columns_with_na.items():
        print(f"- {col}: {count} missing values")
    print(
        "These may be expected (e.g., customs dates for domestic) or require further investigation."
    )
else:
    print(
        "\n--- FINAL REPORT: No missing values remaining in the DataFrame. All handled! ---"
    )

print(f"\nFinal shape of the processed DataFrame: {filtered_df.shape}")


--- FINAL REPORT: Columns with remaining missing values ---
- DT_CUSTOMS_OFFICE_INBOUND_DATE: 7109 missing values
- DT_CUSTOMS_OFFICE_OUTBOUND_DATE: 7108 missing values
- GN_CUSTOMER_SALES_CHANNEL_NAME: 706 missing values
These may be expected (e.g., customs dates for domestic) or require further investigation.

Final shape of the processed DataFrame: (9470, 111)


## 10. Saving Processed Data (Optional)
This section is a placeholder for saving the fully processed DataFrame to a new file (e.g., CSV or Parquet) for use in modeling or further analysis.

In [63]:
# Example: Save the processed DataFrame to a CSV file
output_file_path = "./data/processed_data.csv"
try:
    filtered_df.to_csv(output_file_path, index=False)
    print(f"\nProcessed DataFrame successfully saved to: {output_file_path}")
except Exception as e:
    print(f"\nError saving DataFrame: {e}")

# Example: Save to Parquet for better performance and type preservation
output_parquet_path = "./data/processed_data.parquet"
try:
    filtered_df.to_parquet(output_parquet_path, index=False)
    print(f"\nProcessed DataFrame successfully saved to: {output_parquet_path}")
except Exception as e:
    print(f"\nError saving DataFrame to Parquet: {e}")

print(
    "\nData processing and feature engineering complete. Uncomment saving code if needed."
)


Processed DataFrame successfully saved to: ./data/processed_data.csv

Processed DataFrame successfully saved to: ./data/processed_data.parquet

Data processing and feature engineering complete. Uncomment saving code if needed.
